## 1. Import Qiskit 

In [1]:
from qiskit import QuantumCircuit

## 2. Generate Alice's bits and basis

In [2]:
import numpy as np
import bb84_helper 

n = 30 # number of qubits

bits = np.random.randint(0, 2, n)
basis = np.random.randint(0, 2, n)
print('Bits', bits)
print('Basis', basis)
states = []

for i in range(n):
    state = bb84_helper.bit_basis_to_bb84(bits[i], basis[i])
    states.append(state)

print('States', states)

Bits [1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1 0 1 0 1 1]
Basis [0 1 0 1 0 0 1 1 0 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1]
States ['1', '-', '0', '-', '0', '1', '-', '-', '1', '+', '-', '1', '-', '1', '1', '0', '0', '-', '-', '1', '1', '0', '-', '0', '-', '0', '1', '+', '1', '-']


## 3. Create BB84 States in a circuit

In [3]:
def create_bb84_circuit(bits, basis):
    if len(bits) != len(basis):
        raise ValueError('Bits and basis must have the same length')
    
    n = len(bits)
    qc = QuantumCircuit(n, n)
    for i in range(n):
        if basis[i] == 0:
            if bits[i] == 0:
                qc = bb84_helper.standard(qc, i)
            else:
                qc = bb84_helper.standard_flip(qc, i)
        else:
            if bits[i] == 0:
                qc = bb84_helper.diagonal(qc, i)
            else:
                qc = bb84_helper.diagonal_flip(qc, i)
    return qc

qc = create_bb84_circuit(bits, basis)


## 4. Bob Guesses states used by Alice

In [4]:
guesses = np.random.randint(0, 2, n)
print('Guesses', guesses)

Guesses [0 0 0 0 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 0 0 0 0 1 0 1 1 1 1 0]


## 5. Apply gates based on Bob's guesses

In [5]:
qc.barrier()
# If bob guesses the diagonal basis, apply the Hadamard gate
for i in range(n):
    if guesses[i] == 1:
        qc.h(i)

qc.measure_all(add_bits=False)


# qc.draw(output='mpl')

## 6. Measure Results

In [6]:
from qiskit import transpile
from qiskit_aer import AerSimulator

def run_circuit(circuit):
    simulator = AerSimulator()
    circ = transpile(qc, simulator)

    job = simulator.run(circ, shots=1)
    return job.result(), circ

result, circ = run_circuit(qc)
print('Result', result)
counts = result.get_counts(circ)
print(counts)
# Running one shot, take first index
# reverse the key to get the order of the qubits
bobs_measurements = np.array([int(key) for key in list(counts.keys())[0][::-1]])
print('Bob\'s measurements', bobs_measurements)


Result Result(backend_name='aer_simulator', backend_version='0.14.2', qobj_id='', job_id='c6faf0dd-63a9-4faa-812f-4f8134ef99f9', success=True, results=[ExperimentResult(shots=1, success=True, meas_level=2, data=ExperimentResultData(counts={'0x241f74c3': 1}), header=QobjExperimentHeader(creg_sizes=[['c', 30]], global_phase=0.0, memory_slots=30, n_qubits=30, name='circuit-166', qreg_sizes=[['q', 30]], metadata={}), status=DONE, seed_simulator=4090730646, metadata={'batched_shots_optimization': False, 'required_memory_mb': 16384, 'method': 'statevector', 'active_input_qubits': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'device': 'CPU', 'remapped_qubits': False, 'num_qubits': 30, 'num_clbits': 30, 'time_taken': 15.521301, 'sample_measure_time': 0.2927336, 'input_qubit_map': [[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 5], [6, 6], [7, 7], [8, 8], [9, 9], [10, 10], [11, 11], [12, 12], [13, 13], [14, 14], [15, 15], [16, 16],

## 7. Compare Basis

In [7]:
print(f"Alice's basis: {basis}")
print(f"Bob's basis: {guesses}")

def compare_bases(basis, guesses):
    return np.sum(basis == guesses)


correct_bases = compare_bases(basis, guesses)
print(f"Number of correct bases: {correct_bases} ({correct_bases/n*100}%)")

def discard_nonmatchingbases(bits, bases, guesses):
    alices_key_bits = np.array([], dtype=int)
    bobs_key_bits = np.array([], dtype=int)
    new_bases = np.array([], dtype=int)
    for i in range(len(bits)):
        if bases[i] == guesses[i]:
            alices_key_bits = np.append(alices_key_bits, [bits[i]])
            bobs_key_bits = np.append(bobs_key_bits, [bobs_measurements[i]])
            new_bases = np.append(new_bases, [bases[i]])

    return alices_key_bits, bobs_key_bits, new_bases

alice_key, bob_key, new_bases = discard_nonmatchingbases(bits, basis, guesses)
print('Alice\'s key', alice_key)
print('Bob\'s key', bob_key)
print('New bases', new_bases)

Alice's basis: [0 1 0 1 0 0 1 1 0 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1]
Bob's basis: [0 0 0 0 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 0 0 0 0 1 0 1 1 1 1 0]
Number of correct bases: 15 (50.0%)
Alice's key [1 0 1 1 1 1 1 1 0 1 1 1 1 0 0]
Bob's key [1 0 1 1 1 1 1 1 0 1 1 1 1 0 0]
New bases [0 0 1 1 1 1 0 0 0 1 1 0 0 0 1]


## Security - Check for eavesdropping

### Choose random bits to share publicly

In [8]:
# take 10% of measured bits at random
n = len(alice_key)

indices = np.random.choice(n, int(n*0.25), replace=False)
print('Indices', indices)

alice_key_sample = alice_key[indices]
bob_key_sample = bob_key[indices]

print('Alice\'s key sample', alice_key_sample)
print('Bob\'s key sample', bob_key_sample)

Indices [ 9  0 14]
Alice's key sample [1 1 0]
Bob's key sample [1 1 0]


### Compare bits for interference

In [9]:
# Error threshold to account of noise
# range 0.0 to 1.0
err_threshold = 0.1 # 10%

def compare_keys(alice_key, bob_key):
    return np.sum(alice_key == bob_key)

correct_keys = compare_keys(alice_key_sample, bob_key_sample)
corr_perc = correct_keys/len(alice_key_sample)*100
if corr_perc > 1 - err_threshold:
    print(f"Keys match: {correct_keys} matches ({corr_perc}%)")
else:
    print(f"Keys do not match: {correct_keys} matches ({corr_perc}%)")

Keys match: 3 matches (100.0%)
